In [309]:
import pandas as pd

In [310]:
#load inmate data as df
inmate_df = pd.read_excel('TDCJ Data Set March 2020.xlsx')
inmate_df.head()
inmate_df['County'] = inmate_df['County'].str.replace('Lasalle', 'La Salle')
inmate_df['County'] = inmate_df['County'].str.replace('LaSalle', 'La Salle')
# df5 = inmate_df.groupby(['Current Facility']).count()
# df5.head(100)

In [311]:
#read in census data on the population of each county in Texas 
county_df = pd.read_csv('tx_county_pop_census.csv')
county_df['NAME'] = county_df['NAME'].str.replace(' County, Texas', '')
county_df = county_df.drop([0,255])
county_df.rename(columns={'NAME':'County', 'P1_001N':'total_population'}, inplace=True)
county_df['County'] = county_df['County'].str.upper()
county_df.head()

,GEO_ID,County,total_population,P1_002N,P1_003N,P1_004N,P1_005N,P1_006N,P1_007N,P1_008N,...,P1_062N,P1_063N,P1_064N,P1_065N,P1_066N,P1_067N,P1_068N,P1_069N,P1_070N,P1_071N
1,0500000US48001,ANDERSON,57922,54327,34922,11577,357,392,13,7066,...,0,2,0,2,0,0,0,0,3,3
2,0500000US48003,ANDREWS,18610,14756,10909,237,220,112,0,3278,...,0,3,0,2,0,1,0,0,0,0
3,0500000US48005,ANGELINA,86395,78097,54887,13098,563,919,42,8588,...,0,3,0,3,0,0,0,0,1,1
4,0500000US48007,ARANSAS,23830,20580,17996,264,245,501,17,1557,...,0,7,1,4,0,2,0,0,2,2
5,0500000US48009,ARCHER,8560,8011,7551,32,108,19,0,301,...,0,2,0,0,0,0,0,2,0,0


In [312]:
#read in data from TDCJ on where each prison is located 
prison_df = pd.read_csv('tx_prison_county.csv')
prison_df['County'] = prison_df['County'].str.replace('Duvall','Duval')
prison_df.head()

,Unit,Operator,Sex,Type,Region,City,County,Notes,Unnamed: 8
0,Allred,CID,Male,Prison,V,Iowa Park,Wichita,NaN,NaN
1,B. Moore,MTC,Male,Private Prison,Private,Overton,Rusk,NaN,NaN
2,Baten,CID,Male,Prison,V,Pampa,Gray,Closed/merged with Jordan Unit,NaN
3,Bell,MTC,Male,Private Prison,Private,Cleveland,Liberty,NaN,NaN
4,Beto,CID,Male,Prison,II,Tennessee Colony,Anderson,NaN,NaN


In [313]:
# prison_names1 = prison_df['Unit'].sort_values()

In [314]:
# prison_names2 = sorted(inmate_df['Current Facility'].unique())

In [315]:
# names = []
# for i in prison_names1:
#     i = i.upper()
#     names.append(i)
# prison_names1 = names

# names2 = []
# for j in prison_names2:
#     j = j.upper()
#     names2.append(j)
    
# prison_names2 = names2

In [316]:
# print(prison_names1)

In [317]:
# print(prison_names2)

In [318]:
# for i in prison_names1:
#     if i not in prison_names2:
#         print(f'{i} is not in the inmate data')
#     else:
#         pass
# print('\br')
# for j in prison_names2:
#     if j not in prison_names1:
#         print(f'{j} is not in the TDCJ prison names')
#     else:
#         pass

In [319]:
inmate_df['Current Facility'] = inmate_df['Current Facility'].str.upper()
inmate_df['County'] = inmate_df['County'].str.upper()

In [320]:
prison_df['Unit'] = prison_df['Unit'].str.upper()
prison_df['County'] = prison_df['County'].str.upper()

In [321]:
#COLEMAN = LOCKHART (OR LOCKHART PPT)
#MECHLER = TULIA
#MEMORIAL = DARRINGTON
#WAINWRIGHT = EASTHAM
#Kegans was shut down in '16, hence no inmates
inmate_df['Current Facility'] = inmate_df['Current Facility'].replace({'LOCKHART':'COLEMAN', 
 'LOCKHART PPT':'COLEMAN', 
 'TULIA':'MECHLER',
 'DARRINGTON':'MEMORIAL',
 'EASTHAM':'WAINWRIGHT'
})

In [322]:
inmate_df.rename(columns = {'Current Facility':'current_facility', 'County':'county_charged'}, inplace=True)
prison_df.rename(columns = {'Unit': 'current_facility', 'Gender':'prison_gender'}, inplace=True)
df = pd.merge(inmate_df, prison_df, on ='current_facility', how ="left")

df2 = pd.merge(df, county_df, on='County', how="left")

df2['County'].replace('DUVALL','DUVAL')
df2['county_charged'] = df2['county_charged'].str.replace('DUVALL','DUVAL')
df2['County'] = df2['County'].str.replace('LASALLE', 'LA SALLE')
df2['County'] = df2['County'].str.replace('LASALLE', 'LA SALLE')
#Googled this on TDCJ website to confirm
df2['county_charged']= df2['county_charged'].str.replace('TN','HARRIS')
#Googled this guy, and it looks like he is out on a bench warrant and thus shouldnt even be in our data in the first place
df2 = df2[df2['TDCJ Number'] != 2311086]
df2.reset_index(drop=True)
df2.head()

,SID Number,TDCJ Number,Name,current_facility,Gender,Race,DOB,Projected Release,Maximum Sentence Date,Parole Eligibility Date,...,P1_062N,P1_063N,P1_064N,P1_065N,P1_066N,P1_067N,P1_068N,P1_069N,P1_070N,P1_071N
0,8848100,1760548,"ZYTA,PAMELA GAIL",CRAIN,F,W,1969-08-27,9999-01-01 00:00:00,9999-01-01 00:00:00,2017-07-29 00:00:00,...,0,13,10,3,0,0,0,0,3,3
1,50666793,2198666,"ZWELLING,JASON",HAMILTON,M,W,1988-12-08,2020-05-01 01:30:00,2022-03-05 00:00:00,2018-08-20 00:00:00,...,0,3,0,3,0,0,0,0,0,0
2,2060724,485451,"ZWARST,GERALD PIETER",HUNTSVILLE,M,W,1954-07-09,9999-01-01 00:00:00,9999-01-01 00:00:00,2007-06-27 00:00:00,...,0,2,0,2,0,0,0,0,0,0
3,6515305,1931368,"ZUROVETZ,TODD",CLEMENTS,M,W,1985-08-08,2036-08-01 00:00:00,2037-04-23 00:00:00,2018-02-05 00:00:00,...,0,1,0,1,0,0,0,0,0,0
4,2118310,605015,"ZUROVETZ,LARRY J.",WYNNE,M,W,1957-12-18,2025-07-31 00:00:00,2025-07-31 00:00:00,1999-04-30 00:00:00,...,0,2,0,2,0,0,0,0,0,0


In [323]:
#create dict with county as key and original population as value
total_pop = list(county_df['total_population'])
county = list(county_df['County'].str.upper())
pop_dict = dict(zip(county, total_pop))

for i,j in pop_dict.items():
    pop_dict[i] = int(j)
print(pop_dict)

{'ANDERSON': 57922, 'ANDREWS': 18610, 'ANGELINA': 86395, 'ARANSAS': 23830, 'ARCHER': 8560, 'ARMSTRONG': 1848, 'ATASCOSA': 48981, 'AUSTIN': 30167, 'BAILEY': 6904, 'BANDERA': 20851, 'BASTROP': 97216, 'BAYLOR': 3465, 'BEE': 31047, 'BELL': 370647, 'BEXAR': 2009324, 'BLANCO': 11374, 'BORDEN': 631, 'BOSQUE': 18235, 'BOWIE': 92893, 'BRAZORIA': 372031, 'BRAZOS': 233849, 'BREWSTER': 9546, 'BRISCOE': 1435, 'BROOKS': 7076, 'BROWN': 38095, 'BURLESON': 17642, 'BURNET': 49130, 'CALDWELL': 45883, 'CALHOUN': 20106, 'CALLAHAN': 13708, 'CAMERON': 421017, 'CAMP': 12464, 'CARSON': 5807, 'CASS': 28454, 'CASTRO': 7371, 'CHAMBERS': 46571, 'CHEROKEE': 50412, 'CHILDRESS': 6664, 'CLAY': 10218, 'COCHRAN': 2547, 'COKE': 3285, 'COLEMAN': 7684, 'COLLIN': 1064465, 'COLLINGSWORTH': 2652, 'COLORADO': 20557, 'COMAL': 161501, 'COMANCHE': 13594, 'CONCHO': 3303, 'COOKE': 41668, 'CORYELL': 83093, 'COTTLE': 1380, 'CRANE': 4675, 'CROCKETT': 3098, 'CROSBY': 5133, 'CULBERSON': 2188, 'DALLAM': 7115, 'DALLAS': 2613539, 'DAWSON':

In [324]:
for i in range(0, len(df2)):
    try:
            pop_dict[(df2['County'][i])] -= 1
            pop_dict[(df2['county_charged'][i])] += 1
    except:
        print(i)

31450


In [325]:
adjusted_df = pd.DataFrame(list(pop_dict.items()), columns=['County', 'Adjusted Population'])
adjusted_df.head(100)

,County,Adjusted Population
0,ANDERSON,45273
1,ANDREWS,18710
2,ANGELINA,86264
3,ARANSAS,23998
4,ARCHER,8584
...,...,...
95,HALL,2884
96,HAMILTON,8308
97,HANSFORD,5305
98,HARDEMAN,3600


In [326]:
df3 = pd.merge(df2, adjusted_df, on='County', how="left")
df3.head()

,SID Number,TDCJ Number,Name,current_facility,Gender,Race,DOB,Projected Release,Maximum Sentence Date,Parole Eligibility Date,...,P1_063N,P1_064N,P1_065N,P1_066N,P1_067N,P1_068N,P1_069N,P1_070N,P1_071N,Adjusted Population
0,8848100,1760548,"ZYTA,PAMELA GAIL",CRAIN,F,W,1969-08-27,9999-01-01 00:00:00,9999-01-01 00:00:00,2017-07-29 00:00:00,...,13,10,3,0,0,0,0,3,3,75972
1,50666793,2198666,"ZWELLING,JASON",HAMILTON,M,W,1988-12-08,2020-05-01 01:30:00,2022-03-05 00:00:00,2018-08-20 00:00:00,...,3,0,3,0,0,0,0,0,0,234191
2,2060724,485451,"ZWARST,GERALD PIETER",HUNTSVILLE,M,W,1954-07-09,9999-01-01 00:00:00,9999-01-01 00:00:00,2007-06-27 00:00:00,...,2,0,2,0,0,0,0,0,0,63857
3,6515305,1931368,"ZUROVETZ,TODD",CLEMENTS,M,W,1985-08-08,2036-08-01 00:00:00,2037-04-23 00:00:00,2018-02-05 00:00:00,...,1,0,1,0,0,0,0,0,0,114824
4,2118310,605015,"ZUROVETZ,LARRY J.",WYNNE,M,W,1957-12-18,2025-07-31 00:00:00,2025-07-31 00:00:00,1999-04-30 00:00:00,...,2,0,2,0,0,0,0,0,0,63857


In [327]:
df3.to_csv('tx_inmate_reallocate.csv', index=False)

df4 = pd.merge(county_df, adjusted_df, on='County', how='left')
df4 = df4.drop(df4.iloc[:, 3:73], axis = 1)
df4 = df4.drop(columns=['GEO_ID'], axis=1)
df4.to_csv('tx_county_pop_adjusted.csv', index=False)

In [328]:
election_df = pd.read_csv('tx_election_results_2020.csv')
election_df['County'] = election_df['County'].str.replace('LASALLE', 'LA SALLE')
df5 = pd.merge(df4, election_df, on='County', how='left')
df5.to_csv('tx_pop_adjusted_election.csv', index=False)